In [2]:
import numpy as np
from scipy.optimize import curve_fit
from lagged_coherence import *
import pandas as pd
import scipy

In [3]:
def sigmoid(x, x0, k):
    L=-1
    b=1
    y = L / (1 + np.exp(-k*(x-x0))) + b
    return (y)

In [8]:
f=np.load('../output/sims/oscillation/sim_results.npz')
freqs=f['freqs']
lcs_classic=f['lcs_classic']
lcs_hilbert=f['lcs_hilbert']
psds=f['psds']

trials=100
lags=np.arange(1,6.5,.5)
osc_f=np.arange(10,55,5)

classic_rmse = np.zeros((len(osc_f),trials))*np.nan
hilbert_rmse = np.zeros((len(osc_f),trials))*np.nan
classic_std = np.zeros((len(osc_f),trials))*np.nan
hilbert_std = np.zeros((len(osc_f),trials))*np.nan

for f_idx in range(len(osc_f)):
    for n in range(trials):
        lc_classic = lcs_classic[f_idx,n,:,:]
        lc_hilbert = lcs_hilbert[f_idx,n,:,:]

        f_psd_classic = np.nanmean(lc_classic,axis=1)
        classic_std[f_idx,n] = np.nanstd(f_psd_classic)
        f_psd_classic = f_psd_classic/np.nanmax(f_psd_classic)

        f_psd_hilbert = np.nanmean(lc_hilbert,axis=1)
        hilbert_std[f_idx,n] = np.nanstd(f_psd_hilbert)
        f_psd_hilbert = f_psd_hilbert/np.nanmax(f_psd_hilbert)

        f_psd = psds[f_idx,n,:]
        f_psd = f_psd/np.max(f_psd)

        classic_rmse[f_idx,n] = np.sqrt(np.nansum((f_psd_classic-f_psd)**2))
        hilbert_rmse[f_idx,n] = np.sqrt(np.nansum((f_psd_hilbert-f_psd)**2))
        

# Create DataFrame for classic
df_classic = pd.DataFrame({
    'frequency': np.tile(osc_f, trials),
    'trial': np.repeat(np.arange(trials), len(osc_f)),
    'rmse': classic_rmse.ravel(),
    'std': classic_std.ravel(),
    'algorithm': 'classic'
})

# Create DataFrame for hilbert
df_hilbert = pd.DataFrame({
    'frequency': np.tile(osc_f, trials),
    'trial': np.repeat(np.arange(trials), len(osc_f)),
    'rmse': hilbert_rmse.ravel(),
    'std': hilbert_std.ravel(),
    'algorithm': 'hilbert'
})

# Concatenate the two DataFrames
df_combined = pd.concat([df_classic, df_hilbert])

# Save to CSV
output_path = '../output/sim_oscillations.csv'
df_combined.to_csv(output_path, index=False)
        

f=np.load('../output/sims/burst_dur/sim_results.npz')
freqs=f['freqs']
lcs_classic=f['lcs_classic']
lcs_hilbert=f['lcs_hilbert']
psds=f['psds']
brst_d=f['brst_d']
brst_d_c=f['brst_d_c']
brst_n=f['brst_n']

trials=100
lags=np.arange(1,20.5,.5)
brst_f=np.arange(20,55,5)

d_bursts=np.zeros((len(brst_f),trials))
d_c_bursts=np.zeros((len(brst_f),trials))
x0s=np.zeros((len(brst_f),trials))
ks=np.zeros((len(brst_f),trials))

for f_idx in range(len(brst_f)):
    f=brst_f[f_idx]
    lc_f_idx=np.argmin(np.abs(freqs-f))
    
    for i in range(trials):
        lc_hilbert = lcs_hilbert[f_idx,i,:,:]
        d_bursts[f_idx,i]=brst_d[f_idx,i]
        d_c_bursts[f_idx,i]=brst_d_c[f_idx,i]

        b_lc_hilbert=lc_hilbert[lc_f_idx,:]
        p0 = [np.median(lags),3]
        popt, pcov = curve_fit(sigmoid, lags, b_lc_hilbert, p0, method='lm',maxfev=100000)
        x0s[f_idx,i]=popt[0]
        ks[f_idx,i]=popt[1]
    
# Create DataFrame
df_burst = pd.DataFrame({
    'burst_d': d_bursts.ravel(),
    'burst_d_c': d_c_bursts.ravel(),
    'x0': x0s.ravel(),
    'k': ks.ravel(),
    'freq': np.tile(brst_f, trials),
    'trial': np.repeat(np.arange(trials), len(brst_f))
})

# Save to CSV
output_path_burst = '../output/sim_burst_duration.csv'
df_burst.to_csv(output_path_burst, index=False)

f=np.load('../output/sims/burst_num/sim_results.npz')
freqs=f['freqs']
lcs_classic=f['lcs_classic']
lcs_hilbert=f['lcs_hilbert']
psds=f['psds']
brst_n=f['brst_n']

trials=100
lags=np.arange(1,20.5,.5)
brst_f=np.arange(20,55,5)

n_bursts=np.zeros((len(brst_f),trials))
x0s=np.zeros((len(brst_f),trials))
ks=np.zeros((len(brst_f),trials))

for f_idx in range(len(brst_f)):
    f=brst_f[f_idx]
    lc_f_idx=np.argmin(np.abs(freqs-f))
    
    for i in range(trials):
        lc_hilbert = lcs_hilbert[f_idx,i,:,:]
        n_bursts[f_idx,i]=brst_n[f_idx,i]

        b_lc_hilbert=lc_hilbert[lc_f_idx,:]
        p0 = [np.median(lags),3]
        popt, pcov = curve_fit(sigmoid, lags, b_lc_hilbert, p0, method='lm',maxfev=100000)
        x0s[f_idx,i]=popt[0]
        ks[f_idx,i]=popt[1]

df_burst = pd.DataFrame({
    'burst_n': n_bursts.ravel(),
    'x0': x0s.ravel(),
    'k': ks.ravel(),
    'freq': np.tile(brst_f, trials),
    'trial': np.repeat(np.arange(trials), len(brst_f))
})

# Save to CSV
output_path_burst = '../output/sim_burst_number.csv'
df_burst.to_csv(output_path_burst, index=False)   